In [1]:
import os
from openai import OpenAI

token = os.environ["GITHUB_TOKEN"]
# print(token)
# print(os.environ["OPENAI_API_KEY"])
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)


In [2]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [3]:
persist_directory = "docs/chroma2"

In [4]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

C:\Users\m.landoulsi\AppData\Local\Temp\ipykernel_14248\1143856407.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [5]:
print(vectordb._collection.count())

150


In [6]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [7]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [8]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [9]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [10]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

### Addressing Diversity: Maximum marginal relevance

In [11]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [12]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people c'

In [13]:
docs_ss[1].page_content[:100]

'into his office and he said, "Oh, professor, professor, thank you so much for your \nmachine learning'

In [14]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [15]:
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people c'

In [16]:
docs_mmr[1].page_content[:100]

'reasonable choice on many problems, but you can actually plug in other functions as \nwell. Did I men'

### Addressing Specificity: working with metadata

In [17]:
question = "what did they say about regression in the third lecture?"

In [18]:
docs = vectordb.similarity_search(
    question,
    k=5,
    filter={"source":"MachineLearning-Lecture03.pdf"}
)

In [19]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'MachineLearning-Lecture03.pdf'}
{'page': 14, 'source': 'MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'MachineLearning-Lecture03.pdf'}
{'page': 2, 'source': 'MachineLearning-Lecture03.pdf'}
{'page': 13, 'source': 'MachineLearning-Lecture03.pdf'}


### Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [20]:
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [21]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `MachineLearning-Lecture01.pdf`, `MachineLearning-Lecture02.pdf`, or `MachineLearning-Lecture03.pdf`",
        type="string"
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer"
    )
]

In [22]:
document_content_description = "Lecture notes"
llm = OpenAI(model="gpt-4o",temperature=0.0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)


In [23]:
question = "what did they say about regression in the third lecture?"

In [28]:
docs = retriever.get_relevant_documents(question)

In [29]:
for d in docs:
    print(d.metadata)

{'page': 2, 'source': 'MachineLearning-Lecture03.pdf'}
{'page': 10, 'source': 'MachineLearning-Lecture03.pdf'}
{'page': 11, 'source': 'MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'MachineLearning-Lecture03.pdf'}


### Compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text. 

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. 

In [30]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [31]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [32]:
llm = OpenAI(model="gpt-4o",temperature=0.0)
compressor = LLMChainExtractor.from_llm(llm)

In [35]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [36]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

>>>
those homeworks will be done in either MATLAB or in Octave, which is sort of — I 
know some people call it a free version of MATLAB, which it sort of is, sort of isn't.  
So I guess for those of you that haven't seen MATLAB before, and I know most of you 
have, MATLAB is I guess part of the programming language that makes it very easy to 
write codes using matrices, to write code for numerical routines, to move data around, to 
plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of 
learning algorithms.  
And in case some of you want to work on your own home computer or something if you 
don't have a MATLAB license, for the purposes of this class, there's also — [inaudible] 
write that down [inaudible] MATLAB — there' s also a software package called Octave 
that you can download for free off the Internet. And it has somewhat fewer features than 
MATLAB, but it's free, and for the purposes of this class, it will work for just about

### Combining various techniques


In [37]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type="mmr")
)

In [38]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

>>>
those homeworks will be done in either MATLAB or in Octave, which is sort of — I 
know some people call it a free version of MATLAB, which it sort of is, sort of isn't.  
So I guess for those of you that haven't seen MATLAB before, and I know most of you 
have, MATLAB is I guess part of the programming language that makes it very easy to 
write codes using matrices, to write code for numerical routines, to move data around, to 
plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of 
learning algorithms.  
And in case some of you want to work on your own home computer or something if you 
don't have a MATLAB license, for the purposes of this class, there's also — [inaudible] 
write that down [inaudible] MATLAB — there' s also a software package called Octave 
that you can download for free off the Internet. And it has somewhat fewer features than 
MATLAB, but it's free, and for the purposes of this class, it will work for just about

### Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [39]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [40]:
# Load PDF
loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [ ]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)


In [43]:
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [ ]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

In [44]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(metadata={}, page_content="yourselves. You can also come and talk to me or the TAs if you want to brainstorm ideas \nwith us.  \nOkay. So one more organizational question. I'm curious, how many of you know \nMATLAB? Wow, cool, quite a lot. Okay. So as part of the — act ually how many of you \nknow Octave or have used Octave? Oh, okay, much smaller number.  \nSo as part of this class, especially in the homeworks, we'll ask you to implement a few \nprograms, a few machine learning algorithms as part of the homeworks. And most of  those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn't.  \nSo I guess for those of you that haven't seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it's 